In [3]:
import os
import sys
import time
import numpy as np
import imgaug 

In [4]:
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
from coco.coco import CocoDataset

Using TensorFlow backend.


In [5]:
#COCO_PRETRAINED_WEIGHTS_PATH = os.path.abspath("mask_rcnn_coco.h5")
DATASET_PATH = os.path.abspath("DATASETS/")
TRAIN_ANNOTATIONS_PATH = os.path.abspath("DATASETS/annotations/instances_train2017.json")
LOGS_PATH = os.path.abspath("LOGS")

In [6]:
class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"
    
    IMAGES_PER_GPU = 2 
    #the model fits only once on my 6GB GTX1060. 
    #fits twice on 12GB Tesla P100
    
    NUM_CLASSES = 1 + 80  #80 COCO classes + BG class

In [7]:
coco_train_annotatons = COCO(TRAIN_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=14.13s)
creating index...
index created!


In [8]:
#Train dataset
dataset_train = CocoDataset()
dataset_train.load_coco(dataset_dir=DATASET_PATH, subset="train", year=2017, class_ids=None,
                        class_map=None, return_coco=False, auto_download=False)
dataset_train.prepare()



# Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(dataset_dir=DATASET_PATH, subset="val", year=2017, auto_download=False)
dataset_val.prepare()

loading annotations into memory...
Done (t=14.30s)
creating index...
index created!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


In [12]:
import numpy as np

augmentation = imgaug.augmenters.Fliplr(0.5)


LEARNING_RATES = np.linspace(0.005, 0.0001, 5)
np.append(LEARNING_RATES, 0.02) #add the original learning rate from the paper

histories = []

for lr in LEARNING_RATES:
    
    config = CocoConfig()

    config.LEARNING_RATE = lr
    #config.LEARNING_RATE = 0.0025
    config.LEARNING_MOMENTUM = 0.9

    # Weight decay regularization
    config.WEIGHT_DECAY = 0.0001
    
    print("TRAINING with, LR: {}, DECAY: {}".format(config.LEARNING_RATE, config.WEIGHT_DECAY))
    
    # Create model
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=LOGS_PATH)
        
    # Training - Stage 1
    print("Training network heads")
    hist1 = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=1,
                layers='heads',
                augmentation=augmentation)

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    hist2 = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=8,
                layers='4+',
                augmentation=augmentation)

#     # Training - Stage 3
#     # Fine tune all layers
#     print("Fine tune all layers")
#     hist3 = model.train(dataset_train, dataset_val,
#                 learning_rate=config.LEARNING_RATE / 5,
#                 epochs=1,
#                 layers='all',
#                 augmentation=augmentation)

    histories.append({'model': model, 'hist1': hist1, 'hist2': hist2, 'lr': config.LEARNING_RATE, 'decay': config.WEIGHT_DECAY})

TRAINING with, LR: 0.01, DECAY: 0.0001
Training network heads

Starting at epoch 0. LR=0.01

Checkpoint Path: /home/013855803/CMPE-297-homework/LOGS/coco20190413T2258/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        

/home/013855803/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/013855803/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1
1000/1000 [==============================] - 1608s 2s/step - loss: 5.1994 - rpn_class_loss: 0.7011 - rpn_bbox_loss: 2.2694 - mrcnn_class_loss: 0.1804 - mrcnn_bbox_loss: 1.7372 - mrcnn_mask_loss: 0.3113 - val_loss: 1.9885 - val_rpn_class_loss: 0.6952 - val_rpn_bbox_loss: 1.2933 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Fine tune Resnet stage 4 and up

Starting at epoch 1. LR=0.01

Checkpoint Path: /home/013855803/CMPE-297-homework/LOGS/coco20190413T2258/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c

1000/1000 [==============================] - 1435s 1s/step - loss: nan - rpn_class_loss: 0.4763 - rpn_bbox_loss: nan - mrcnn_class_loss: 2.5024 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_class_loss: 0.3183 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 2.5024 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 1210s 1s/step - loss: nan - rpn_class_loss: 0.4045 - rpn_bbox_loss: nan - mrcnn_class_loss: 2.5024 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_class_loss: 0.3003 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 2.5024 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 1459s 1s/step - loss: nan - rpn_class_loss: 0.4677 - rpn_bbox_loss: nan - mrcnn_class_loss: 2.5024 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_cl

Epoch 2/10
1000/1000 [==============================] - 1521s 2s/step - loss: nan - rpn_class_loss: 0.7010 - rpn_bbox_loss: nan - mrcnn_class_loss: 3.7059 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_class_loss: 0.7059 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 3.7059 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 1397s 1s/step - loss: nan - rpn_class_loss: 0.7020 - rpn_bbox_loss: nan - mrcnn_class_loss: 3.7059 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_class_loss: 0.6994 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 3.7059 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 1228s 1s/step - loss: nan - rpn_class_loss: 0.7030 - rpn_bbox_loss: nan - mrcnn_class_loss: 3.7059 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan -

Epoch 2/10
1000/1000 [==============================] - 1829s 2s/step - loss: nan - rpn_class_loss: 0.4739 - rpn_bbox_loss: nan - mrcnn_class_loss: 2.8662 - mrcnn_bbox_loss: 0.0315 - mrcnn_mask_loss: 0.0083 - val_loss: nan - val_rpn_class_loss: 0.4889 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 3.8658 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 1254s 1s/step - loss: nan - rpn_class_loss: 0.4656 - rpn_bbox_loss: nan - mrcnn_class_loss: 3.8658 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn_class_loss: 0.3679 - val_rpn_bbox_loss: nan - val_mrcnn_class_loss: 3.8658 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 1495s 1s/step - loss: nan - rpn_class_loss: 0.4657 - rpn_bbox_loss: nan - mrcnn_class_loss: 3.8658 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: nan - val_rpn


Epoch 2/10
1000/1000 [==============================] - 1699s 2s/step - loss: 1.7907 - rpn_class_loss: 0.2888 - rpn_bbox_loss: 0.7748 - mrcnn_class_loss: 0.1658 - mrcnn_bbox_loss: 0.2367 - mrcnn_mask_loss: 0.3245 - val_loss: 3.6952 - val_rpn_class_loss: 0.6169 - val_rpn_bbox_loss: 1.1763 - val_mrcnn_class_loss: 0.3119 - val_mrcnn_bbox_loss: 0.8586 - val_mrcnn_mask_loss: 0.7313
Epoch 3/10
1000/1000 [==============================] - 1416s 1s/step - loss: 1.8563 - rpn_class_loss: 0.1149 - rpn_bbox_loss: 0.6098 - mrcnn_class_loss: 0.2741 - mrcnn_bbox_loss: 0.3374 - mrcnn_mask_loss: 0.5201 - val_loss: 6.2208 - val_rpn_class_loss: 0.4547 - val_rpn_bbox_loss: 2.7657 - val_mrcnn_class_loss: 1.1925 - val_mrcnn_bbox_loss: 0.7106 - val_mrcnn_mask_loss: 1.0972
Epoch 4/10
1000/1000 [==============================] - 1402s 1s/step - loss: 1.9453 - rpn_class_loss: 0.1189 - rpn_bbox_loss: 0.7580 - mrcnn_class_loss: 0.2592 - mrcnn_bbox_loss: 0.3320 - mrcnn_mask_loss: 0.4772 - val_loss: 7.4165 - val_r

Epoch 2/10
 945/1000 [===========================>..] - ETA: 1:21 - loss: 1.9663 - rpn_class_loss: 0.2019 - rpn_bbox_loss: 0.5375 - mrcnn_class_loss: 0.2641 - mrcnn_bbox_loss: 0.4550 - mrcnn_mask_loss: 0.5079

ResourceExhaustedError: OOM when allocating tensor with shape[400,256,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_9/SGD/gradients/mrcnn_mask_4/convolution_grad/Conv2DBackpropInput}} = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@train...propFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_9/SGD/gradients/mrcnn_mask_4/convolution_grad/Conv2DBackpropInput-0-VecPermuteNHWCToNCHW-LayoutOptimizer, mrcnn_mask_4/kernel/read, training_9/SGD/gradients/mrcnn_mask_4/Sigmoid_grad/SigmoidGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_9/add_5/_23793}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/re

In [16]:
#loss visualization
import matplotlib.pyplot as plt

plt.plot(histories[0]['hist2'].history['loss'])
plt.show()
hist2.history
hist1.history

IndexError: list index out of range